In [1]:
import pandas as pd

In [ ]:
import pandas as pd
import os
# Start coding here... 

def extract_data(file_path: str, file_name: str, source_file_type:str, custom_headers: list = None, **kwargs) -> pd.DataFrame:
    """
    Extract data from a CSV or Excel file.

    Parameters:
        file_name (str): Name of the file.
        file_path (str): Path to the file.
        source_file_type (str): Type of the file ('csv', 'xlsx' or 'json').
        custom_headers (list): List columns to set in the DataFrame.
        **kwargs: Additional arguments to pass to the pandas read functions.

    Returns:
        pd.DataFrame: DataFrame containing the extracted data.

    Raises:
        FileNotFoundError: If the file does not exist.
        ValueError: If the file type is not supported.
        Exception: If an error occurs while reading the file.
    """
    full_path = os.path.join(file_path, file_name)
    
    if not os.path.exists(full_path):
        raise FileNotFoundError(f"The file {full_path} does not exist.")
    
    try:
        if source_file_type == "csv":
            df = pd.read_csv(full_path, **kwargs)
        elif source_file_type == "xlsx":
            df =  pd.read_excel(full_path, header=None, names=custom_headers, **kwargs) if custom_headers else pd.read_excel(full_path, **kwargs)
        elif source_file_type == "json":
            df = pd.read_json(full_path).T.reset_index()
            df.columns = custom_headers
        else:
            raise ValueError("Unsupported file type. Please use 'csv', 'xlsx' or 'json'.")
        print(f"File {full_path} successfully read as a {source_file_type} file.")
        return df
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        raise

df_office_addresses = extract_data(file_path="datasets", file_name="office_addresses.csv", source_file_type="csv")

df_employee_addresses = extract_data(file_path="datasets", file_name="employee_information.xlsx", source_file_type="xlsx", sheet_name=0)

df_employee_contacts = extract_data(file_path="datasets", file_name="employee_information.xlsx", source_file_type="xlsx", sheet_name="emergency_contacts", custom_headers=["employee_id", "last_name", "first_name", "emergency_contact", "emergency_contact_number", "relationship"])

df_employee_role_team_salary = extract_data(file_path="datasets", file_name="employee_roles.json", source_file_type="json", custom_headers=["employee_id", "title", "monthly_salary", "team"])
df_employee_role_team_salary.head()

In [ ]:
def merge_dataframes(dataframes: list, keys: list, how: str) -> pd.DataFrame:
    """
    Merge multiple DataFrames with different keys.

    Parameters:
        dfs (list): List of DataFrames to merge.
        keys (list): List of keys (column names) to join on, one for each DataFrame.
        how (str): Type of merge to perform (default is 'outer').

    Returns:
        pd.DataFrame: Merged DataFrame.
    """
    merged_df = dataframes[0]
    for i in range(1, len(dataframes)):
        merged_df = pd.merge(merged_df, dataframes[i], how=how , left_on=keys[0], right_on=keys[i])
    return merged_df

employee_information = merge_dataframes(dataframes=[df_employee_addresses,df_employee_contacts,df_employee_role_team_salary], keys=["employee_id", "employee_id", "employee_id"], how="inner")

employees_final = pd.merge(employee_information, office_addresses, how="left",right_on=['office_country'],
    left_on=['employee_country'])

office_columns = [col for col in employees_final.columns if col.startswith('office')]

employees_final[office_columns] = employees_final[office_columns].applymap(lambda x: 'Remote' if pd.isna(x) else x)

employees_final = employees_final.set_index("employee_id")

columns = ["employee_first_name", "employee_last_name", "employee_country", "employee_city", "employee_street", "employee_street_number", "emergency_contact", "emergency_contact_number", "relationship", "monthly_salary", "team", "title", "office", "office_country", "office_city", "office_street", "office_street_number"]

employees_final = employees_final[columns]

employees_final.head()